In [ ]:
import rasterio

def inspect_geotiff_metadata(tif_path):
    with rasterio.open(tif_path) as src:
        print(f" File: {tif_path}")
        print(" CRS:", src.crs)
        print(" Transform (Affine):", src.transform)
        print(" Bounds:", src.bounds)
        print(" Bands:", src.count)
        print(" Data type:", src.dtypes)
        print(" NoData value:", src.nodata)
        print(" Width × Height (pixels):", src.width, "×", src.height)
        
        for i in range(1, src.count + 1):
            band = src.read(i)
            print(f" Band {i}: min = {band.min()}, max = {band.max()}")
        print("-" * 50)


In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point


def extract_geotiff_features(df, lat_col, lon_col, tif_dict):
    """
    Extract features from geotiff files
    """
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=[Point(xy) for xy in zip(df[lon_col], df[lat_col])],
        crs="EPSG:4326"
    )

    for feature_name, tif_path in tif_dict.items():
        print(f"Extracting: {feature_name} from {tif_path}")
        with rasterio.open(tif_path) as src:
            band = src.read(1) 
            nodata = src.nodata
            transform = src.transform

            def get_value(row):
                try:
                    row_idx, col_idx = src.index(row[lon_col], row[lat_col])
                    val = band[row_idx, col_idx]
                    return None if (val == nodata or val < -99900) else val
                except:
                    return None

            gdf[feature_name] = gdf.apply(get_value, axis=1)

    return gdf.drop(columns=["geometry"])



## 1.Gravity Data Extraction

We extracted key gravity features from three high-resolution GeoTIFF grids provided by Geoscience Australia, covering different aspects of the Earth's crustal density variations. These datasets are georeferenced, gridded products representing gravity anomalies across Australia and were sampled at each training point’s latitude and longitude.

Each GeoTIFF contains one raster band (float32) with a known NoData value, which we masked during processing.

| Feature Name          | Source File                          | Description |
|-----------------------|---------------------------------------|-------------|
| `gravity_iso_residual` | `Gravmap2016-grid-grv_ir.tif`         | Isostatic residual anomaly — reflects shallow density anomalies after isostatic correction. Highlights features like intrusions or potential ore bodies. |
| `gravity_cscba`        | `Gravmap2019-grid-grv_cscba.tif`      | Complete Spherical Cap Bouguer Anomaly (CSCBA) — represents full Bouguer anomaly adjusted with a spherical cap correction, providing insight into regional deep crustal structures. |
| `gravity_cscba_1vd`    | `Gravmap2019-grid-grv_cscba_1vd.tif`  | First vertical derivative of CSCBA — emphasizes structural boundaries such as faults and intrusive contacts. Useful for enhancing linear geological features. |



**Processing Notes**

- All values were extracted using rasterio and matched to sample locations via geographic coordinates.

- Any values equal to or less than the NoData threshold (e.g. -99999.0) were excluded.

- The final gravity features were appended to the training dataset as new columns.

- No normalization or transformation was applied at this stage, leaving it flexible for later preprocessing (e.g., standardization).

In [1]:
gravity_iso_residual = "../data/raw/Dataset/GA/Gravity/Gravmap2016-grid-grv_ir.tif"
gravity_cscba = "../data/raw/Dataset/GA/Gravity/Gravmap2019-grid-grv_cscba.tif"
gravity_cscba_1vd = "../data/raw/Dataset/GA/Gravity/Gravmap2019-grid-grv_cscba_1vd.tif"

In [3]:
inspect_geotiff_metadata(gravity_iso_residual)
inspect_geotiff_metadata(gravity_cscba)
inspect_geotiff_metadata(gravity_cscba_1vd)

 File: ../data/raw/Dataset/GA/Gravity/Gravmap2016-grid-grv_ir.tif
 CRS: GEOGCS["GDA94",DATUM["unnamed",SPHEROID["unnamed",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
 Transform (Affine): | 0.01, 0.00, 112.50|
| 0.00,-0.01,-10.10|
| 0.00, 0.00, 1.00|
 Bounds: BoundingBox(left=112.4975, bottom=-44.268, right=154.173, top=-10.098)
 Bands: 1
 Data type: ('float32',)
 NoData value: -99999.0
 Width × Height (pixels): 4903 × 4020
 Band 1: min = -99999.0, max = 1167.243896484375
--------------------------------------------------
 File: ../data/raw/Dataset/GA/Gravity/Gravmap2019-grid-grv_cscba.tif
 CRS: GEOGCS["GDA94",DATUM["unnamed",SPHEROID["unnamed",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
 Transform (Affine): | 0.00, 0.00, 111.00|
| 0.00,-0.00,-8.00|
| 0.00, 0.00, 1.00|

In [5]:
df = pd.read_csv("../data/processed/final_samples_balanced.csv")

tif_paths = {
    "gravity_iso_residual": "../data/raw/Dataset/GA/Gravity/Gravmap2016-grid-grv_ir.tif",
    "gravity_cscba": "../data/raw/Dataset/GA/Gravity/Gravmap2019-grid-grv_cscba.tif",
    "gravity_cscba_1vd": "../data/raw/Dataset/GA/Gravity/Gravmap2019-grid-grv_cscba_1vd.tif"
}

df_with_gravity = extract_geotiff_features(
    df=df,
    lat_col="LATITUDE",
    lon_col="LONGITUDE",
    tif_dict=tif_paths
)

df_with_gravity.to_csv("../data/processed/final_samples_with_gravity.csv", index=False)
print("Saved as final_samples_with_gravity.csv")

Extracting: gravity_iso_residual from ../data/raw/Dataset/GA/Gravity/Gravmap2016-grid-grv_ir.tif
Extracting: gravity_cscba from ../data/raw/Dataset/GA/Gravity/Gravmap2019-grid-grv_cscba.tif
Extracting: gravity_cscba_1vd from ../data/raw/Dataset/GA/Gravity/Gravmap2019-grid-grv_cscba_1vd.tif
Saved as final_samples_with_gravity.csv


## 2.Magnetic Features Extraction

We extracted magnetic anomaly features from the AWAGS_MAG_2019 dataset provided by Geoscience Australia. These GeoTIFF grids represent upward continued Total Magnetic Intensity (TMI) data that have been Reduced to the Pole (RTP) and smoothed at varying vertical levels. Each upward continuation depth highlights magnetic responses at a specific crustal level and is valuable for identifying both shallow and deep-seated geological structures related to porphyry copper systems.

Each raster file contains:

- A single float32 band of TMI values (in nanoTesla, approximate)

- A geographic coordinate system (GDA94)

- A high spatial resolution grid (~0.00085° per pixel)

- A consistent NoData value of -99999.0 (which we masked out during processing)

The following magnetic features were extracted by sampling each raster at the sample point coordinates (LATITUDE, LONGITUDE) and appended to the dataset:

| Feature Name      | Upward Continuation Depth | Interpretation                          |
|-------------------|----------------------------|------------------------------------------|
| `mag_uc_1_2km`    | 1–2 km                     | Emphasizes shallow magnetic sources      |
| `mag_uc_2_4km`    | 2–4 km                     | Intermediate-scale structures            |
| `mag_uc_4_8km`    | 4–8 km                     | Balanced view of deeper features         |
| `mag_uc_8_12km`   | 8–12 km                    | Highlights deep crustal trends           |
| `mag_uc_12_16km`  | 12–16 km                   | Emphasizes regional tectonic patterns    |


These features provide multi-scale magnetic context, allowing the machine learning model to learn from both surface and subsurface magnetic patterns.

We recommend:

- Applying Z-score standardization to each magnetic feature;

- Masking or imputing NoData values prior to modeling.


In [6]:
mag_uc_1_2km = "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC1km2kmRes-AWAGS_MAG_2019.tif"
mag_uc_2_4km = "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC2km4kmRes-AWAGS_MAG_2019.tif"
mag_uc_4_8km = "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC4km8kmRes-AWAGS_MAG_2019.tif"
mag_uc_8_12km = "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC8km12kmRes-AWAGS_MAG_2019.tif"
mag_uc_12_16km = "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC12km16kmRes-AWAGS_MAG_2019.tif"

In [ ]:
inspect_geotiff_metadata(mag_uc_1_2km)
inspect_geotiff_metadata(mag_uc_2_4km)
inspect_geotiff_metadata(mag_uc_4_8km)
inspect_geotiff_metadata(mag_uc_8_12km)
inspect_geotiff_metadata(mag_uc_12_16km)

 File: ../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC1km2kmRes-AWAGS_MAG_2019.tif
 CRS: GEOGCS["GDA94",DATUM["unnamed",SPHEROID["unnamed",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
 Transform (Affine): | 0.00, 0.00, 111.00|
| 0.00,-0.00,-9.03|
| 0.00, 0.00, 1.00|
 Bounds: BoundingBox(left=110.99980000000001, bottom=-43.930550000000004, right=154.66430000000003, top=-9.026150000000001)
 Bands: 1
 Data type: ('float32',)
 NoData value: -99999.0
 Width × Height (pixels): 51370 × 41064
 Band 1: min = -99999.0, max = 6989.29443359375
--------------------------------------------------
 File: ../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC2km4kmRes-AWAGS_MAG_2019.tif
 CRS: GEOGCS["GDA94",DATUM["unnamed",SPHEROID["unnamed",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORT

In [7]:
tif_paths_mag = {
    "mag_uc_1_2km": "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC1km2kmRes-AWAGS_MAG_2019.tif",
    "mag_uc_2_4km": "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC2km4kmRes-AWAGS_MAG_2019.tif",
    "mag_uc_4_8km": "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC4km8kmRes-AWAGS_MAG_2019.tif",
    "mag_uc_8_12km": "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC8km12kmRes-AWAGS_MAG_2019.tif",
    "mag_uc_12_16km": "../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC12km16kmRes-AWAGS_MAG_2019.tif"
}

In [8]:
df_with_gravity = pd.read_csv("../data/processed/final_samples_with_gravity.csv")
df_with_magnetics = extract_geotiff_features(
    df=df_with_gravity,
    lat_col="LATITUDE",
    lon_col="LONGITUDE",
    tif_dict=tif_paths_mag
)

df_with_magnetics.to_csv("../data/processed/final_samples_with_gravity_magnetics.csv", index=False)
print("Saved: final_samples_with_gravity_magnetics.csv")

Extracting: mag_uc_1_2km from ../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC1km2kmRes-AWAGS_MAG_2019.tif
Extracting: mag_uc_2_4km from ../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC2km4kmRes-AWAGS_MAG_2019.tif
Extracting: mag_uc_4_8km from ../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC4km8kmRes-AWAGS_MAG_2019.tif
Extracting: mag_uc_8_12km from ../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC8km12kmRes-AWAGS_MAG_2019.tif
Extracting: mag_uc_12_16km from ../data/raw/Dataset/GA/Magnetic/Magmap2019-grid-tmi_rtp_upcon-UC12km16kmRes-AWAGS_MAG_2019.tif
Saved: final_samples_with_gravity_magnetics.csv


## 3.Radiometric Features Extraction

We extracted radiometric features from filtered and ratio-based GeoTIFF datasets published by Geoscience Australia. These grids represent the concentrations of naturally occurring radioactive elements (Potassium, Thorium, and Uranium), and their ratios, which are useful indicators of:

- Rock types and weathering profiles,

- Hydrothermal alteration zones,

- Potential ore-related geochemical anomalies.

Each GeoTIFF raster is georeferenced and contains a single float32 band, with a NoData value of -99999.0. All values were sampled at each training point's geographic coordinates and added as new columns.

| Feature Name         | Source File                                             | Description |
|----------------------|----------------------------------------------------------|-------------|
| `radio_K_pct`        | `Radmap2019-grid-k_conc-Filtered-AWAGS_RAD_2019.tif`     | Filtered potassium concentration (%). Often elevated in felsic rocks or potassic alteration zones. |
| `radio_Th_ppm`       | `Radmap2019-grid-th_conc-Filtered-AWAGS_RAD_2019.tif`    | Filtered thorium concentration (ppm). Typically enriched in residual soils and weathered felsic rocks. |
| `radio_U_ppm`        | `Radmap2019-grid-u_conc-Filtered-AWAGS_RAD_2019.tif`     | Filtered uranium concentration (ppm). Sensitive to groundwater leaching, may indicate hydrothermal systems. |
| `radio_Th_K_ratio`   | `Radmap2019-grid-thk_ratio-AWAGS_RAD_2019.tif`           | Thorium to potassium ratio. Useful for identifying weathered versus fresh bedrock. |
| `radio_U_K_ratio`    | `Radmap2019-grid-uk_ratio-AWAGS_RAD_2019.tif`            | Uranium to potassium ratio. May highlight uranium-enriched zones relative to host lithology. |
| `radio_U_Th_ratio`   | `Radmap2019-grid-uth_ratio-AWAGS_RAD_2019.tif`           | Uranium to thorium ratio. Often elevated in alteration zones or areas of uranium mobility. |

**Processing Notes**
- All rasters were sampled using bilinear or nearest-neighbor interpolation, depending on resolution alignment.

- NoData values were excluded (-99999.0) and handled during model preprocessing.

- Features are preserved in their raw units for modeling flexibility (e.g., log-transform or standardize if needed).

In [20]:
radio_k_pct = "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-k_conc-Filtered-AWAGS_RAD_2019.tif"
radio_Th_ppm = "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-th_conc-Filtered-AWAGS_RAD_2019.tif"
radio_U_ppm = "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-u_conc-Filtered-AWAGS_RAD_2019.tif"
radio_Th_K_ratio = "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-thk_ratio-AWAGS_RAD_2019.tif"
radio_U_K_ratio = "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-uk_ratio-AWAGS_RAD_2019.tif"
radio_U_Th_ratio = "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-uth_ratio-AWAGS_RAD_2019.tif"

In [21]:
inspect_geotiff_metadata(radio_k_pct)
inspect_geotiff_metadata(radio_Th_ppm)
inspect_geotiff_metadata(radio_U_ppm)
inspect_geotiff_metadata(radio_Th_K_ratio)
inspect_geotiff_metadata(radio_U_K_ratio)
inspect_geotiff_metadata(radio_U_Th_ratio)

 File: ../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-k_conc-Filtered-AWAGS_RAD_2019.tif
 CRS: GEOGCS["GDA94",DATUM["unnamed",SPHEROID["unnamed",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
 Transform (Affine): | 0.00, 0.00, 112.90|
| 0.00,-0.00,-9.00|
| 0.00, 0.00, 1.00|
 Bounds: BoundingBox(left=112.899, bottom=-43.761, right=153.671, top=-8.999)
 Bands: 1
 Data type: ('float32',)
 NoData value: -99999.0
 Width × Height (pixels): 40772 × 34762
 Band 1: min = -99999.0, max = 1370.40234375
--------------------------------------------------
 File: ../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-th_conc-Filtered-AWAGS_RAD_2019.tif
 CRS: GEOGCS["GDA94",DATUM["unnamed",SPHEROID["unnamed",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
 Transform (Affine): | 0.00, 0.00, 

In [22]:
tif_paths_radio = {
    "radio_K_pct": "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-k_conc-Filtered-AWAGS_RAD_2019.tif",
    "radio_Th_ppm": "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-th_conc-Filtered-AWAGS_RAD_2019.tif",
    "radio_U_ppm": "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-u_conc-Filtered-AWAGS_RAD_2019.tif",
    "radio_U_K_ratio": "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-uk_ratio-AWAGS_RAD_2019.tif",
    "radio_U_Th_ratio": "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-uth_ratio-AWAGS_RAD_2019.tif",
    "radio_Th_K_ratio": "../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-thk_ratio-AWAGS_RAD_2019.tif"
}

In [23]:
df_with_radiometric = extract_geotiff_features(
    df=df_with_magnetics,
    lat_col="LATITUDE",
    lon_col="LONGITUDE",
    tif_dict=tif_paths_radio
)

Extracting: radio_K_pct from ../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-k_conc-Filtered-AWAGS_RAD_2019.tif
Extracting: radio_Th_ppm from ../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-th_conc-Filtered-AWAGS_RAD_2019.tif
Extracting: radio_U_ppm from ../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-u_conc-Filtered-AWAGS_RAD_2019.tif
Extracting: radio_U_K_ratio from ../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-uk_ratio-AWAGS_RAD_2019.tif
Extracting: radio_U_Th_ratio from ../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-uth_ratio-AWAGS_RAD_2019.tif
Extracting: radio_Th_K_ratio from ../data/raw/Dataset/GA/Radiometric/Radmap2019-grid-thk_ratio-AWAGS_RAD_2019.tif


In [24]:
df_with_radiometric.to_csv("../data/processed/final_samples_with_gravity_magnetics_radiometric.csv", index=False)
print("Saved: final_samples_with_all_geophysical_features.csv")

Saved: final_samples_with_all_geophysical_features.csv


## 4.Model Input Samples

We constructed the final dataset for model training by integrating spatial coordinates, sample source tags, and a comprehensive suite of geophysical features derived from national-scale raster datasets provided by Geoscience Australia. The input features were selected based on their geological significance and relevance to porphyry copper mineralization.

| Feature Group        | Fields Included                                                                                  |
|----------------------|--------------------------------------------------------------------------------------------------|
| **Spatial Coordinates** | `LONGITUDE`, `LATITUDE` — used for mapping, visualization, or spatial regularization              |
| **Sample Source**        | `SOURCE` — denotes origin of the sample (positive, other deposit, blank area)                     |
| **Gravity Features**     | `gravity_iso_residual`, `gravity_cscba`, `gravity_cscba_1vd`                                     |
| **Magnetic Features**    | `mag_uc_1_2km`, `mag_uc_2_4km`, `mag_uc_4_8km`, `mag_uc_8_12km`, `mag_uc_12_16km`               |
| **Radiometric Features** | `radio_K_pct`, `radio_Th_ppm`, `radio_U_ppm`, `radio_Th_K_ratio`, `radio_U_K_ratio`, `radio_U_Th_ratio` |
| **Label (Target)**       | `LABEL` — binary class indicating presence or absence of porphyry copper mineralization           |


**Notes**
- All geophysical features were extracted from GeoTIFF raster grids using precise geographic sampling.

- NoData values (-99999.0) were masked prior to modeling.

- No transformations were applied at this stage; downstream models may include normalization, imputation, or feature engineering.

- The resulting dataset was saved as model_input_samples.csv.




In [25]:
features = [
    "LONGITUDE", "LATITUDE", "SOURCE", 
    "gravity_iso_residual", "gravity_cscba", "gravity_cscba_1vd",
    "mag_uc_1_2km", "mag_uc_2_4km", "mag_uc_4_8km", "mag_uc_8_12km", "mag_uc_12_16km",
    "radio_K_pct", "radio_Th_ppm", "radio_U_ppm",
    "radio_Th_K_ratio", "radio_U_K_ratio", "radio_U_Th_ratio",
    "LABEL" 
]

df_model = df_with_radiometric[features]
df_model.to_csv("../data/processed/model_input_samples.csv", index=False)
